In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, BaggingClassifier
from sklearn.svm import SVC
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.metrics import accuracy_score, classification_report

2024-06-30 11:06:47.956432: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-30 11:06:47.956559: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-30 11:06:48.146251: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
train = pd.read_csv('/kaggle/input/playground-series-s4e6/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s4e6/test.csv')
sample_submission = pd.read_csv('/kaggle/input/playground-series-s4e6/sample_submission.csv')

In [3]:
train.head()

,id,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,0,1,1,1,9238,1,1,126.0,1,1,...,0,6,7,6,12.428571,0,11.1,0.6,2.02,Graduate
1,1,1,17,1,9238,1,1,125.0,1,19,...,0,6,9,0,0.000000,0,11.1,0.6,2.02,Dropout
2,2,1,17,2,9254,1,1,137.0,1,3,...,0,6,0,0,0.000000,0,16.2,0.3,-0.92,Dropout
3,3,1,1,3,9500,1,1,131.0,1,19,...,0,8,11,7,12.820000,0,11.1,0.6,2.02,Enrolled
4,4,1,1,2,9500,1,1,132.0,1,19,...,0,7,12,6,12.933333,0,7.6,2.6,0.32,Graduate


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76518 entries, 0 to 76517
Data columns (total 38 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   id                                              76518 non-null  int64  
 1   Marital status                                  76518 non-null  int64  
 2   Application mode                                76518 non-null  int64  
 3   Application order                               76518 non-null  int64  
 4   Course                                          76518 non-null  int64  
 5   Daytime/evening attendance                      76518 non-null  int64  
 6   Previous qualification                          76518 non-null  int64  
 7   Previous qualification (grade)                  76518 non-null  float64
 8   Nacionality                                     76518 non-null  int64  
 9   Mother's qualification                 

In [5]:
train.describe()

,id,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,...,Curricular units 1st sem (without evaluations),Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP
count,76518.000000,76518.000000,76518.000000,76518.000000,76518.000000,76518.000000,76518.000000,76518.000000,76518.000000,76518.000000,...,76518.00000,76518.000000,76518.000000,76518.000000,76518.000000,76518.000000,76518.000000,76518.000000,76518.000000,76518.000000
mean,38258.500000,1.111934,16.054419,1.644410,9001.286377,0.915314,3.658760,132.378766,1.226600,19.837633,...,0.05796,0.137053,5.933414,7.234468,4.007201,9.626085,0.062443,11.520340,1.228218,-0.080921
std,22088.988286,0.441669,16.682337,1.229645,1803.438531,0.278416,8.623774,10.995328,3.392183,15.399456,...,0.40849,0.933830,1.627182,3.503040,2.772956,5.546035,0.462107,2.653375,1.398816,2.251382
min,0.000000,1.000000,1.000000,0.000000,33.000000,0.000000,1.000000,95.000000,1.000000,1.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.600000,-0.800000,-4.060000
25%,19129.250000,1.000000,1.000000,1.000000,9119.000000,1.000000,1.000000,125.000000,1.000000,1.000000,...,0.00000,0.000000,5.000000,6.000000,1.000000,10.000000,0.000000,9.400000,0.300000,-1.700000
50%,38258.500000,1.000000,17.000000,1.000000,9254.000000,1.000000,1.000000,133.100000,1.000000,19.000000,...,0.00000,0.000000,6.000000,7.000000,5.000000,12.142857,0.000000,11.100000,1.400000,0.320000
75%,57387.750000,1.000000,39.000000,2.000000,9670.000000,1.000000,1.000000,140.000000,1.000000,37.000000,...,0.00000,0.000000,6.000000,9.000000,6.000000,13.244048,0.000000,12.700000,2.600000,1.790000
max,76517.000000,6.000000,53.000000,9.000000,9991.000000,1.000000,43.000000,190.000000,109.000000,44.000000,...,12.00000,19.000000,23.000000,33.000000,20.000000,18.000000,12.000000,16.200000,3.700000,3.510000


## Preprocessing

#### 1- Feature Selection

In [6]:
train.drop(columns=['Nacionality'], inplace=True, axis=1)

In [7]:
test.drop(columns=['Nacionality'], inplace=True, axis=1)

#### 2- Define Features X and Target y

In [8]:
X = train.iloc[:, 1:-1]
y = train.iloc[:, -1]

#### 3- Data Scaling

In [9]:
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

#### 4- Data Encoding

In [10]:
one_hot_encoder = OneHotEncoder()
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

#### 5- Data Splitting

In [11]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)
print(X_train.shape)

(68866, 35)


## Machine Learning Algorithms

In [13]:
svc = SVC()
svc.fit(X_train, y_train)

SVC()

In [16]:
y_pred = svc.predict(X_val)


# Model Evaluation
accuracy = accuracy_score(y_val, y_pred)
accuracy_percentage = round(accuracy * 100, 3)
train_accuracy = round(svc.score(X_train, y_train) * 100, 3)
print(f'Training Accuracy: {train_accuracy}%')
print(f'Testing Accuracy: {accuracy_percentage}%')

Training Accuracy: 81.993%
Testing Accuracy: 81.051%


In [14]:
catboost = CatBoostClassifier(learning_rate=0.001,random_seed=42, verbose=0, iterations=2000)
catboost.fit(X_train, y_train)
y_pred = catboost.predict(X_val)


# Model Evaluation
accuracy = accuracy_score(y_val, y_pred)
accuracy_percentage = round(accuracy * 100, 3)
train_accuracy = round(catboost.score(X_train, y_train) * 100, 3)
print(f'Training Accuracy: {train_accuracy}%')
print(f'Testing Accuracy: {accuracy_percentage}%')
# accuracy_df.loc[len(accuracy_df)] = ['CatBoostClassifier',train_accuracy,accuracy_percentage]

Training Accuracy: 81.6%
Testing Accuracy: 81.168%


In [15]:
lgbm = LGBMClassifier(learning_rate=0.01, n_estimators=700, random_state=42)
lgbm.fit(X_train, y_train)
y_pred = lgbm.predict(X_val)


# Model Evaluation
accuracy = accuracy_score(y_val, y_pred)
accuracy_percentage = round(accuracy * 100, 3)
train_accuracy = round(lgbm.score(X_train, y_train) * 100, 3)
print(f'Training Accuracy: {train_accuracy}%')
print(f'Testing Accuracy: {accuracy_percentage}%')
# accuracy_df.loc[len(accuracy_df)] = ['LGBMClassifier',train_accuracy,accuracy_percentage]

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.020060 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1350
[LightGBM] [Info] Number of data points in the train set: 68866, number of used features: 35
[LightGBM] [Info] Start training from score -1.106016
[LightGBM] [Info] Start training from score -1.634523
[LightGBM] [Info] Start training from score -0.746379
Training Accuracy: 84.243%
Testing Accuracy: 83.129%


#### Prepare the Submission File 

In [17]:
test

,id,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Mother's qualification,Father's qualification,...,Curricular units 1st sem (without evaluations),Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP
0,76518,1,1,1,9500,1,1,141.0,3,1,...,0,0,8,0,0,0.000000,0,13.9,-0.3,0.79
1,76519,1,1,1,9238,1,1,128.0,1,19,...,0,0,6,6,6,13.500000,0,11.1,0.6,2.02
2,76520,1,1,1,9238,1,1,118.0,1,19,...,0,0,6,11,5,11.000000,0,15.5,2.8,-4.06
3,76521,1,44,1,9147,1,39,130.0,1,19,...,0,3,8,14,5,11.000000,0,8.9,1.4,3.51
4,76522,1,39,1,9670,1,1,110.0,1,37,...,0,0,6,9,4,10.666667,2,7.6,2.6,0.32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51007,127525,1,1,2,171,1,1,128.0,38,37,...,0,0,0,0,0,0.000000,0,15.5,2.8,-4.06
51008,127526,2,39,1,9119,1,19,133.1,19,37,...,0,0,5,5,0,0.000000,0,9.4,-0.8,-3.12
51009,127527,1,1,1,171,1,1,127.0,1,1,...,0,0,0,0,0,0.000000,0,15.5,2.8,-4.06
51010,127528,1,1,3,9773,1,1,132.0,19,19,...,0,0,6,9,3,13.000000,0,7.6,2.6,0.32


In [18]:
test = test.set_index(test['id'])
test

,id,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Mother's qualification,Father's qualification,...,Curricular units 1st sem (without evaluations),Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP
id,,,,,,,,,,,,,,,,,,,,,
76518,76518,1,1,1,9500,1,1,141.0,3,1,...,0,0,8,0,0,0.000000,0,13.9,-0.3,0.79
76519,76519,1,1,1,9238,1,1,128.0,1,19,...,0,0,6,6,6,13.500000,0,11.1,0.6,2.02
76520,76520,1,1,1,9238,1,1,118.0,1,19,...,0,0,6,11,5,11.000000,0,15.5,2.8,-4.06
76521,76521,1,44,1,9147,1,39,130.0,1,19,...,0,3,8,14,5,11.000000,0,8.9,1.4,3.51
76522,76522,1,39,1,9670,1,1,110.0,1,37,...,0,0,6,9,4,10.666667,2,7.6,2.6,0.32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127525,127525,1,1,2,171,1,1,128.0,38,37,...,0,0,0,0,0,0.000000,0,15.5,2.8,-4.06
127526,127526,2,39,1,9119,1,19,133.1,19,37,...,0,0,5,5,0,0.000000,0,9.4,-0.8,-3.12
127527,127527,1,1,1,171,1,1,127.0,1,1,...,0,0,0,0,0,0.000000,0,15.5,2.8,-4.06


In [20]:
test.drop(columns=['id'], inplace=True, axis=1)
test.head()

,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Mother's qualification,Father's qualification,Mother's occupation,...,Curricular units 1st sem (without evaluations),Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP
id,,,,,,,,,,,,,,,,,,,,,
76518,1,1,1,9500,1,1,141.0,3,1,2,...,0,0,8,0,0,0.000000,0,13.9,-0.3,0.79
76519,1,1,1,9238,1,1,128.0,1,19,4,...,0,0,6,6,6,13.500000,0,11.1,0.6,2.02
76520,1,1,1,9238,1,1,118.0,1,19,4,...,0,0,6,11,5,11.000000,0,15.5,2.8,-4.06
76521,1,44,1,9147,1,39,130.0,1,19,3,...,0,3,8,14,5,11.000000,0,8.9,1.4,3.51
76522,1,39,1,9670,1,1,110.0,1,37,5,...,0,0,6,9,4,10.666667,2,7.6,2.6,0.32


In [21]:
y_pred = lgbm.predict(test)

In [26]:
y_pred

array([0, 2, 2, ..., 0, 2, 0])

In [27]:
y_pred = label_encoder.inverse_transform(y_pred)
y_pred

array(['Dropout', 'Graduate', 'Graduate', ..., 'Dropout', 'Graduate',
       'Dropout'], dtype=object)

In [22]:
sample_submission

,id,Target
0,76518,Graduate
1,76519,Graduate
2,76520,Graduate
3,76521,Graduate
4,76522,Graduate
...,...,...
51007,127525,Graduate
51008,127526,Graduate
51009,127527,Graduate
51010,127528,Graduate


In [28]:
submission = pd.DataFrame({'id': test.index, 'Target': y_pred}, columns=['id', 'Target'])

submission.to_csv('submission.csv', index=False)